# Querying Agents

## Imports

In [ ]:
# General
import os
from dotenv import load_dotenv

# AutoGen
import autogen

# LangChain





# Get API keys from .env file
load_dotenv()

## AutoGen

### Define helper methods

In [37]:
# Define openai configurations
openai_config_list = [
    {
        'model': 'gpt-3.5-turbo',                 # $0.03 for running the first request below (can't get live data, used placeholder data)
        # 'model': 'gpt-4',                           # $0.33 for running the two requests below
        'api_key': os.environ['OPENAI_API_KEY'],
    }
]

### Define the Agent

In [ ]:
# Create AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,                             # Seed for caching/reproducibility
        "config_list": openai_config_list,             # OpenAI API configurations
        "temperature": 0,                       # Randomness, 0 means no randomness
    }                                           # Config for autogen's enhanced inference API
)

# Create UserProxyAgent named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",                   # NEVER, ALWAYS, or TERMINATE (Recommendation; 1 question: NEVER, multiple followup questions: TERMINATE)
    max_consecutive_auto_reply=10,              # Max number of auto replies
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),   # Termination condition
    code_execution_config={
        "work_dir": "coding",                   # Working directory for code execution (where generated files are stored)
        "use_docker": False,                    # Set to True or image name like "python:3" to use docker
    }
)

### Test the Agent

#### Stock price comparison

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
)

Followup question:

In [ ]:
user_proxy.send(
    recipient=assistant,
    message="""Plot a chart of their stock price change YTD and save to stock_price_ytd.png.""",
)

#### Sum all numbers in a range

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""Sum all the numbers from 0 to 10.""",
)

#### Sort array in descending order

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""Sort the array [1, 3, 2, 4, 5, 7] in descending order.""",
)

#### Describe Djikstra's algorithm

In [ ]:
# Assistant receives message from user_proxy
user_proxy.initiate_chat(
    assistant,
    message="""Describe Djikstra's algorithm using at most 200 words.""",
)

## LangChain

### Define helper methods

### Define the Agent

### Test the Agent

#### Sum all numbers in a range

#### Sort array in descending order

#### Describe Djikstra's algorithm